<a href="https://www.kaggle.com/code/fluloeo/rag-prototype?scriptVersionId=295122047" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install "protobuf<5" vllm transformers chromadb langchain -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.6/192.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/89

## Преобразование статей из postgres в векторную бд chroma для прототипирования RAG

я пока не задумывался над тем как там все векторизуется и фрагментируется на чанки

In [2]:
import psycopg2
from kaggle_secrets import UserSecretsClient
import chromadb
from IPython.display import display, Markdown
import numpy as np 
import pandas as pd 
import os
import ast
from tqdm import tqdm
from typing import List, Dict, Tuple
from vllm import LLM, SamplingParams
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableLambda
from transformers import AutoTokenizer
import torch
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


secrets = UserSecretsClient()
db_params = {
    "dbname": "arxivdb",
    "user": secrets.get_secret("DB_USER"),
    "password": secrets.get_secret("DB_PASSWORD"),
    "host": secrets.get_secret("DB_HOST"),
    "port": "5433"  
}

with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cursor:
        print("Выгрузка данных из Postgres...")
        cursor.execute("SELECT id, title, abstract, clean_text FROM arxivdb.public.articles LIMIT 1000")
        rows = cursor.fetchall()
        
client = chromadb.PersistentClient(path="./my_rag_db")

try:
    client.delete_collection(name="scientific_papers")
except:
    pass
collection = client.create_collection(name="scientific_papers")

db_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
    separators=["\n\n", "\n", ". ", " ", ""]
)

all_ids = []
all_documents = []
all_metadatas = []

print("Разбивка на чанки...")
for row in rows:
    article_id, title, abstract, full_text = str(row[0]), row[1], row[2], row[3]
    
    if not full_text or len(full_text) < 10: 
        continue
        
    article_chunks = db_splitter.split_text(full_text)
    
    for i, chunk in enumerate(article_chunks):
        all_ids.append(f"{article_id}_ch_{i}")
        all_documents.append(chunk)
        all_metadatas.append({
            "article_id": article_id, 
            "title": title,
            "abstract": abstract
        })

print(f"Индексация в ChromaDB ({len(all_documents)} фрагментов)...")
batch_size = 1000
for i in tqdm(range(0, len(all_documents), batch_size)):
    collection.add(
        ids=all_ids[i : i + batch_size],
        documents=all_documents[i : i + batch_size],
        metadatas=all_metadatas[i : i + batch_size]
    )

print(f"Готово! Всего в ChromaDB: {collection.count()} записей.")

2026-01-31 12:02:27.896308: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769860948.126533      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769860948.192608      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769860948.756699      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769860948.756741      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769860948.756744      24 computation_placer.cc:177] computation placer alr

/kaggle/input/df_dict_test.csv
Выгрузка данных из Postgres...
Разбивка на чанки...
Индексация в ChromaDB (28183 фрагментов)...


  0%|          | 0/29 [00:00<?, ?it/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:   0%|          | 0.00/79.3M [00:00<?, ?iB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:   1%|          | 629k/79.3M [00:00<00:15, 5.25MiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:   9%|▉         | 7.04M/79.3M [00:00<00:01, 38.7MiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:  21%|██        | 16.4M/79.3M [00:00<00:01, 64.3MiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:  32%|███▏      | 25.6M/79.3M [00:00<00:00, 76.9MiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:  44%|████▍     | 34.7M/79.3M [00:00<00:00, 83.5MiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:  55%|█████▌    | 43.9M/79.3M [00:00<00:00, 87.8MiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:  67%|██████▋   | 53.1M/79.3M [00:00<00:00, 90.6MiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-

Готово! Всего в ChromaDB: 28183 записей.


Проверка

In [3]:
results = collection.get(
    limit=300, 
    include=["metadatas"]
)
unique_articles = {}
for meta in results['metadatas']:
    a_id = meta['article_id']
    title = meta['title']
    if a_id not in unique_articles:
        unique_articles[a_id] = title
    if len(unique_articles) >= 10:
        break
print("--- Список первых 10 уникальных статей в ChromaDB ---")
for i, (a_id, title) in enumerate(unique_articles.items(), 1):
    print(f"{i}. [ID: {a_id}] {title}")

--- Список первых 10 уникальных статей в ChromaDB ---
1. [ID: 2009.07542] perturbation expansions and error bounds for the truncated singular value decomposition
2. [ID: 1908.09022] neural data-to-text generation: a comparison between pipeline and end-to-end architectures
3. [ID: 2208.04846] mining reaction and diffusion dynamics in social activities
4. [ID: 1402.1467] reconstruction models for attractors in the technical and economic processes
5. [ID: 2107.10649] tagrec: automated tagging of questions with hierarchical learning taxonomy
6. [ID: 2010.11939] limitations of autoregressive models and their alternatives
7. [ID: 1207.1746] a generic library for stencil computations
8. [ID: 1804.04385] convergence of a finite volume scheme for a system of interacting species with cross-diffusion
9. [ID: 2009.08008] the boundary element method of peridynamics
10. [ID: 2111.0571] object servoing of differential-drive robots


## Импорты и загрузка модели

In [4]:

model_id = "Qwen/Qwen3-4B-Instruct-2507"
llm = LLM(
    model= model_id,
    tensor_parallel_size=2,  
    max_model_len=16384,     
    gpu_memory_utilization=0.85,
    attention_backend="TRITON_ATTN",
    dtype="float16" 
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

INFO 01-31 12:21:24 [utils.py:261] non-default args: {'dtype': 'float16', 'max_model_len': 16384, 'tensor_parallel_size': 2, 'gpu_memory_utilization': 0.85, 'disable_log_stats': True, 'attention_backend': 'TRITON_ATTN', 'model': 'Qwen/Qwen3-4B-Instruct-2507'}


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

INFO 01-31 12:21:43 [model.py:541] Resolved architecture: Qwen3ForCausalLM
WARNING 01-31 12:21:43 [model.py:1885] Casting torch.bfloat16 to torch.float16.
INFO 01-31 12:21:43 [model.py:1561] Using max model len 16384
INFO 01-31 12:21:44 [scheduler.py:226] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 01-31 12:21:44 [vllm.py:624] Asynchronous scheduling is enabled.


generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

(Worker_TP0 pid=250) INFO 01-31 12:23:56 [backends.py:302] Cache the graph of compile range (1, 8192) for later use
(Worker_TP1 pid=251) INFO 01-31 12:23:57 [backends.py:302] Cache the graph of compile range (1, 8192) for later use
(Worker_TP0 pid=250) INFO 01-31 12:24:08 [backends.py:319] Compiling a graph for compile range (1, 8192) takes 24.87 s
(Worker_TP0 pid=250) INFO 01-31 12:24:08 [monitor.py:34] torch.compile takes 42.07 s in total
(Worker_TP0 pid=250) INFO 01-31 12:24:11 [gpu_worker.py:356] Available KV cache memory: 7.05 GiB
(EngineCore_DP0 pid=226) INFO 01-31 12:24:11 [kv_cache_utils.py:1307] GPU KV cache size: 102,704 tokens
(EngineCore_DP0 pid=226) INFO 01-31 12:24:11 [kv_cache_utils.py:1312] Maximum concurrency for 16,384 tokens per request: 6.27x


Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:05<00:00,  9.37it/s]
Capturing CUDA graphs (decode, FULL):  89%|████████▊ | 31/35 [00:10<00:02,  1.47it/s]

(Worker_TP1 pid=251) INFO 01-31 12:24:28 [custom_all_reduce.py:216] Registering 6278 cuda graph addresses


Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:12<00:00,  2.84it/s]


(Worker_TP0 pid=250) INFO 01-31 12:24:29 [custom_all_reduce.py:216] Registering 6278 cuda graph addresses
(Worker_TP0 pid=250) INFO 01-31 12:24:30 [gpu_model_runner.py:5051] Graph capturing finished in 19 secs, took 0.52 GiB
(EngineCore_DP0 pid=226) INFO 01-31 12:24:30 [core.py:272] init engine (profile, create kv cache, warmup model) took 72.36 seconds
(EngineCore_DP0 pid=226) INFO 01-31 12:24:35 [vllm.py:624] Asynchronous scheduling is enabled.
INFO 01-31 12:24:35 [llm.py:343] Supported tasks: ['generate']


## Проверка запуска модели

In [5]:
# sampling_params = SamplingParams(temperature = 0, max_tokens=512)
# messages = [{"role": "user", "content": "Кратко расскажи о больших языковых моделях."}]

# prompt = tokenizer.apply_chat_template(
#     messages, 
#     tokenize=False, 
#     add_generation_prompt=True
# )

# outputs = llm.generate([prompt], sampling_params)
# for output in outputs:
#     print(f"Ответ: {output.outputs[0].text}\n")

## Минимальная обработка чанков датасета (здесь не используется, так как я взял из бд полные текста статей)

In [6]:
def to_full_text(data_dict):
    """Превращает словарь разделов в единый текст."""
    return "\n\n".join([f"{k}\n{v}" for k, v in data_dict.items()])

def get_token_length(text: str) -> int:
    return len(tokenizer.encode(text))

# def merge_small_chunks_by_tokens(chunks: List[str], min_tokens: int) -> List[str]:
#     """
#     Объединяет чанки, если их длина в ТОКЕНАХ меньше min_tokens.
#     Слияние происходит с наименьшим соседом (по количеству токенов).
#     """
#     processed_chunks = chunks[:]
#     separator = " " 
    
#     i = 0
#     while i < len(processed_chunks):
#         current_chunk = processed_chunks[i]
#         current_len = get_token_length(current_chunk)
#         if current_len >= min_tokens:
#             i += 1
#             continue
#         if len(processed_chunks) == 1:
#             break
#         if i > 0:
#             left_len = get_token_length(processed_chunks[i-1])
#         else:
#             left_len = float('inf')

#         if i < len(processed_chunks) - 1:
#             right_len = get_token_length(processed_chunks[i+1])
#         else:
#             right_len = float('inf')
#         if left_len < right_len:
#             processed_chunks[i-1] = processed_chunks[i-1] + separator + processed_chunks[i]
#             processed_chunks.pop(i)
#             i -= 1 
#         else:
#             processed_chunks[i] = processed_chunks[i] + separator + processed_chunks[i+1]
#             processed_chunks.pop(i+1)
#     return processed_chunks
    
# def merge_chunks_in_dataset(df, min_tokens=100, n=100, n_threshold = 10):
#     data = pd.DataFrame(columns=['chunks'])
#     max_len_chunks = []
#     max_size_chunks = []
#     min_size_chunks = []
#     for i in range(n):
#         data_dict = ast.literal_eval(df['dict_test'].iloc[i])
#         tokens_chunks_list_0 = list(map(lambda x: get_token_length(x), list(data_dict.values())))
#         if len(tokens_chunks_list_0)>n_threshold or min(tokens_chunks_list_0)<min_tokens:
#             final_chunks = merge_small_chunks_by_tokens(list(data_dict.values()), min_tokens=min_tokens)
#             # print(f"\nКонечное количество чанков: {len(final_chunks)}\n")
#             tokens_chunks_list = list(map(lambda x: get_token_length(x), final_chunks))
#             # print('Максимальный размер чанка',max(tokens_chunks_list),'\n')
#             max_len_chunks.append(len(final_chunks))
#             max_size_chunks.append(max(tokens_chunks_list))
#             min_size_chunks.append(min(tokens_chunks_list))
#             data.loc[len(data)] = [final_chunks]
#         else:
#             print("Ничего не меняем, кол-во чанков, мин токенов", len(tokens_chunks_list_0),min(tokens_chunks_list_0))
#             data.loc[len(data)] = [data_dict]
#     print('Максимальное Кол-во чанков из всех статей\n',(max(max_len_chunks)))
#     print('Минимальное Кол-во чанков из всех статей\n',(min(max_len_chunks)))
#     print('Максимальный размер чанка из всех статей\n',max(max_size_chunks))
#     print('Минимальный размер чанка из всех статей\n',min(min_size_chunks))
#     return data
# df = pd.read_csv('/kaggle/input/df-dict-test/df_dict_test.csv')
# data = merge_chunks_in_dataset(df=df, min_tokens=700, n=20,n_threshold = 10)

In [7]:
def fetch_article_by_id(article_id: str) -> Dict:
    """Получает полные данные статьи из PostgreSQL."""
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cur:
            cur.execute("SELECT id, title, abstract, clean_text FROM articles WHERE id = %s", (article_id,))
            row = cur.fetchone()
            if row:
                return {"id": row[0], "title": row[1], "abstract": row[2], "text": row[3]}
    return None

In [8]:
def vllm_gen(prompts, max_new_tokens):
    """Принимает список промптов и лимит токенов."""
    if not prompts: return []
    current_params = SamplingParams(
        temperature=0, 
        max_tokens=max_new_tokens
    )
    
    outputs = llm.generate(prompts, current_params)
    return [output.outputs[0].text for output in outputs]

def process_chunks(pre_parsed_chunks, max_chunk_tokens=3000):
    """
    Проверяет готовые чанки. Если чанк слишком большой, дробит его.
    """
    splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer=tokenizer,
        chunk_size=max_chunk_tokens,
        chunk_overlap=200
    )
    
    final_chunks = []
    for chunk in pre_parsed_chunks:
        token_count = len(tokenizer.encode(chunk))
        if token_count <= max_chunk_tokens:
            final_chunks.append(chunk)
        else:
            sub_chunks = splitter.split_text(chunk)
            final_chunks.extend(sub_chunks)
            
    return final_chunks

def prepare_prompt(text, system_prompt):
    messages = [{"role": "user", "content": f"{system_prompt}:\n\n{text}"}]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

def run_summarization_pipeline(pre_parsed_chunks, p0, p1):
    ready_chunks = process_chunks(pre_parsed_chunks, max_chunk_tokens=3500)

    map_prompts = [prepare_prompt(c, p0) for c in ready_chunks]
    chunk_summaries = gen_chunks_runnable.invoke(map_prompts)
    
    combined_text = "\n".join(chunk_summaries)
    
    reduce_prompt = prepare_prompt(combined_text, p1)
    final_summaries = gen_final_runnable.invoke([reduce_prompt])
    return final_summaries[0]

summaries_list = []
gen_chunks_runnable = RunnableLambda(lambda x: vllm_gen(x, max_new_tokens=512))
gen_final_runnable = RunnableLambda(lambda x: vllm_gen(x, max_new_tokens=1280))
prompt_0 = """Act as a professional Science Editor. 
Your task is to extract key information from this fragment of a scientific paper.
1. Identify specific objectives, technical methodologies, and key data mentioned.
2. Maintain technical terminology and the original language.
3. Be objective: do not add interpretations or info not present in the text.
4. Output format: Use a dense bulleted list of the most important points.
5. Avoid any introductory or concluding remarks—just the facts."""
prompt_1 = """Act as a Senior Science Editor. 
Task: Synthesize a single, coherent executive summary based on the provided fragment summaries.

Structure your response strictly as follows:
- **Goal**: Clear statement of the research objective and problem.
- **Methods**: Summary of the technical approach, experiments, or theoretical framework.
- **Results**: Detailed overview of key findings and performance improvements.
- **Conclusion**: The overall significance and final takeaway.

Requirements:
- Style: Academic, professional, and precise.
- Formulas: Do not use any mathematical formulas or LaTeX. Describe results in words.
- Length: Aim for 500-650 words. Ensure information density is high.
- Integrity: Ensure the text ends with a complete, definitive concluding sentence.
- Do not add any text outside of the headers. Final summary only."""

# for i in range(20): #len(data)
#     torch.cuda.empty_cache()
#     print(f"\n=== Обработка статьи {i} ===")
#     current_article_chunks = data['chunks'].iloc[i] 
#     try:
#         summary = run_summarization_pipeline(current_article_chunks, prompt_0, prompt_1)
#         article = to_full_text(ast.literal_eval(df['dict_test'].iloc[i]))
#         print(f"Статья {i} готова. Итоговых токенов: {len(tokenizer.encode(summary))}")
#         summaries_list.append({
#             'article_id': i,
#             'summary': summary,
#             'abstract': df['abstract'].iloc[i],
#             'article': article
#         })
#     except Exception as e:
#         print(f"Ошибка на статье {i}: {e}")

# pd.DataFrame(summaries_list).to_csv('summaries.csv')
# data_load = pd.read_csv('/kaggle/working/summaries.csv')
# data_load.head()

In [9]:
class ScientificSearch:
    def __init__(self):
        self.intent_prompt = """Analyze the user query. Classify it into one of two categories:
1. SUMMARIZE: The user wants a summary of a specific article (often mentions an ID or asks for a general summary).
2. QA: The user is asking a factual question about scientific topics and needs information from multiple sources.

Answer only with the word 'SUMMARIZE' or 'QA'."""

    def classify_intent(self, query: str) -> str:
        prompt = tokenizer.apply_chat_template([{"role": "user", "content": f"{self.intent_prompt}\nQuery: {query}"}], tokenize=False, add_generation_prompt=True)
        return vllm_gen([prompt], 10)[0].strip().upper()

    def handle_qa(self, query: str):
        """RAG сценарий: поиск по всей базе ChromaDB."""
        results = collection.query(query_texts=[query], n_results=3)
        context = "\n\n".join(results['documents'][0])
        
        qa_prompt = f"Using the following scientific context, answer the question: {query}\n\nContext:\n{context}"
        prompt = tokenizer.apply_chat_template([{"role": "user", "content": qa_prompt}], tokenize=False, add_generation_prompt=True)
        answer = vllm_gen([prompt], 700)[0]
        
        sources = [m['title'] for m in results['metadatas'][0]]
        return answer, sources

    def handle_summarize(self, query: str):
        # 1. Поиск ближайшего чанка
        results = collection.query(query_texts=[query], n_results=1)
        if not results['ids'][0]:
            return "No matching articles found in Vector DB.", []

        # Вытаскиваем статью из метаданных чанка
        meta = results['metadatas'][0][0]
        article_id = meta['article_id']
        title = meta['title']
        
        print(f"[*] Found article: '{title}' (ID: {article_id}). Fetching full text...")
        
        # 2. Получение текста
        article_data = fetch_article_by_id(str(article_id))
        
        if not article_data or not article_data['text']:
            return f"Error: Full text for '{title}' not found in Postgres.", []

        # 3. Суммаризация
        # Используем RecursiveCharacterTextSplitter для нарезки полного текста
        full_text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
        chunks = full_text_splitter.split_text(article_data['text'])
        
        print(f"[*] Summarizing {len(chunks)} chunks...")
        summary = run_summarization_pipeline(chunks, prompt_0, prompt_1)
        
        return summary, [title]

    def run(self, query: str):
        intent = self.classify_intent(query)
        print(f"[*] Detected Intent: {intent}")
        
        if "SUMMARIZE" in intent:
            return self.handle_summarize(query)
        else:
            return self.handle_qa(query)

# --- 4. ЗАПУСК ---
Search = ScientificSearch()

# Пример 1: Вопрос (QA / RAG)
q1 = "How do proximal gradient algorithms handle errors?"
ans1, src1 = Search.run(q1)
display(Markdown(f"\n[QA Answer]: {ans1}\n[Sources]: {src1}"))

# Пример 2: Суммаризация (Summarization)
q2 = "Summarize the paper about style transfer and image robustness."
ans2, src2 = Search.run(q2)
display(Markdown(f"\n[Summary]: {ans2}\n[Article]: {src2}"))

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[*] Detected Intent: QA


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Adding requests:   0%|          | 0/12 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/12 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


[Summary]: **Goal**:  
The research addresses the fundamental challenge of multimodal unsupervised image-to-image translation in the absence of paired domain examples. Existing methods, such as UNIT and CycleGAN, rely on deterministic mappings or pair supervision, failing to capture the full diversity of possible outputs. The core problem lies in the ill-posed nature of translation without supervision, where models struggle to generate diverse, realistic outputs while preserving semantic content and style variations.

**Methods**:  
The proposed framework introduces a stochastic, disentangled latent space model based on a shared content space and domain-specific style spaces. Each image is decomposed into a content code (encoding semantic structure) and a style code (encoding visual appearance such as lighting, weather, or timing). Translation is achieved by extracting the content code from a source image and combining it with a randomly sampled style code from the target domain. The model employs generative adversarial networks (GANs) to align the distribution of generated images with real target domain data. A bidirectional reconstruction loss ensures encoder-decoder consistency, while a style-augmented cycle consistency constraint enforces joint distribution alignment without requiring deterministic mappings. A domain-invariant perceptual loss, computed using VGG features after Instance Normalization, enhances domain invariance and improves realism. The architecture includes content and style encoders with Adaptive Instance Normalization (AdaIN) in the decoder, enabling flexible style transfer.

**Results**:  
The model generates diverse and realistic outputs across multiple domains, outperforming existing unsupervised baselines such as UNIT and CycleGAN. Human preference evaluations on Amazon Mechanical Turk show that MUNIT produces more accurate and visually plausible results than alternatives. Quantitative metrics confirm superior performance: the full MUNIT model achieves the highest Conditional Inception Score (CIS) and Inception Score (IS), indicating both high quality and multimodal diversity. In edge-to-shoes/handbags, animal, and street scene translations, outputs span multiple visual modes (e.g., rainy, snowy, winter, summer) while preserving content. Ablation studies reveal that content and style reconstruction losses are essential—removing either degrades quality or induces mode collapse. The model matches the performance of fully supervised BicycleGAN in diversity and realism, without requiring paired training data.

**Conclusion**:  
This work presents the first unified, unsupervised framework for multimodal image-to-image translation that achieves both high-quality output generation and rich diversity through a disentangled content-style latent space, enabling effective translation without paired examples.
[Article]: ['multimodal unsupervised image-to-image translation']

In [10]:
# Пример 1: Технический вопрос по оптимизации (Статья №9)
q1 = "What is MTL2L and how does it use context for optimization?"
ans1, src1 = Search.run(q1)
display(Markdown(f"### Тест 1: QA (Оптимизация)\n**Запрос:** {q1}\n\n[QA Answer]: {ans1}\n\n**Источники:** {src1}"))

# Пример 2: Вопрос по компьютерному зрению (Статья №4)
q2 = "How does histogram equalization prior help in unsupervised low-light image enhancement?"
ans2, src2 = Search.run(q2)
display(Markdown(f"### Тест 2: QA (Computer Vision)\n**Запрос:** {q2}\n\n[QA Answer]: {ans2}\n\n**Источники:** {src2}"))

# Пример 3: Вопрос по робототехнике и алгоритмам (Статья №7)
q3 = "What are the challenges of gathering myopic robots on a triangular grid under asynchrony?"
ans3, src3 = Search.run(q3)
display(Markdown(f"### Тест 3: QA (Робототехника)\n**Запрос:** {q3}\n\n[QA Answer]: {ans3}\n\n**Источники:** {src3}"))

# --- ПРОВЕРКА СУММАРИЗАЦИИ ---

# Пример 4: Суммаризация статьи про юриспруденцию (Статья №2)
q4 = "Summarize the case study on Chinese codex regarding legal word embeddings."
ans4, src4 = Search.run(q4)
display(Markdown(f"### Тест 4: Суммаризация (Legal NLP)\n**Запрос:** {q4}\n\n[Summary]: {ans4}\n\n**Статья:** {src4}"))

# Пример 5: Суммаризация статьи про поиск кода (Статья №3)
q5 = "Provide a summary of the paper about accelerating code search using deep hashing."
ans5, src5 = Search.run(q5)
display(Markdown(f"### Тест 5: Суммаризация (Software Engineering)\n**Запрос:** {q5}\n\n[Summary]: {ans5}\n\n**Статья:** {src5}"))

# Пример 6: Сложный вопрос про теорию нейросетей (Статья №10)
q6 = "What is the impact of initialization on specialization according to the theory proposed?"
ans6, src6 = Search.run(q6)
display(Markdown(f"### Тест 6: QA (DL Theory)\n**Запрос:** {q6}\n\n[QA Answer]: {ans6}\n\n**Источники:** {src6}"))

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[*] Detected Intent: QA


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[*] Detected Intent: QA


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

### Тест 6: QA (DL Theory)
**Запрос:** What is the impact of initialization on specialization according to the theory proposed?

[QA Answer]: The provided scientific context does **not** directly discuss or propose a theory linking **initialization** to **specialization** in a general or explicit theoretical framework. Therefore, based solely on the given text, **there is no clear or direct answer to the question: "What is the impact of initialization on specialization according to the theory proposed?"**

Let’s break this down carefully:

---

### 1. **What is "initialization" in the context?**
- In **object-oriented programming (OOP)**, initialization refers to the automatic execution of a **constructor** when an object is created. This happens *before* any other method is called, ensuring that object state is set up properly.
- In **Transformers (deep learning models)**, initialization refers to the scaling of parameters (e.g., patchify weights, positional embeddings) such that preactivations remain of order one throughout forward propagation — a principle called **criticality**.
- This ensures stable signal propagation and avoids vanishing or exploding gradients.

---

### 2. **What is "specialization"?**
- The term "specialization" is not explicitly defined or discussed in the context.
- In OOP, specialization might refer to subclassing or creating more specific versions of classes (e.g., a `SavingsAccount` specialized from `Account`).
- In machine learning, it might refer to model specialization (e.g., adapting a general model to a specific task).
- However, **no such concept is formally introduced or linked to initialization** in the provided text.

---

### 3. **Is there a theory linking initialization to specialization?**
- The context describes:
  - How constructors initialize object state (OOP).
  - How parameter initialization in Transformers ensures stable signal propagation (criticality).
- But **no theory connects initialization to specialization** — whether in programming or in machine learning.
- The mention of "object initialization" and "method calls" is about **object lifecycle**, not about **specialization**.
- The discussion of initialization in neural networks is about **stability and signal scaling**, not about how models become specialized.

---

### Conclusion:

> **According to the theory proposed in the context, there is no stated or implied impact of initialization on specialization.**

The provided text discusses:
- The role of constructors in object initialization (OOP),
- The importance of initialization in neural networks to maintain signal stability (criticality),
- But **does not define or explore the concept of specialization**, nor does it propose a theory linking initialization to it.

---

### Final Answer:
**There is no proposed theory in the context that describes the impact of initialization on specialization. The text discusses initialization in the context of object creation and signal stability in neural networks, but does not link it to specialization.**

**Источники:** ['an approach to programming based on concepts', 'effective theory of transformers at initialization', 'traits for correct-by-construction programming']